**Importation des bibliothèques nécessaires**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
import os

# Change the working directory
os.chdir(r'C:\project')

**Chargement des données**

In [ ]:
#chargement des données
df = pd.read_csv("data/avito_cars_clean.csv")

# Afficher les cinq premières lignes des données
df.head()

,annee,boite,carburant,kilometrage,marque,modele,nombre_portres,premiere_main,puissance_fiscale,etat,prix
0,2023,Automatique,Essence,22499.5,Alfa Romeo,Tonale,5,Oui,8,Excellent,306000
1,2018,Manuelle,Diesel,104999.5,Dacia,Duster,5,Oui,6,Excellent,155000
2,2023,Automatique,Diesel,67499.5,Hyundai,Accent,5,Oui,6,Excellent,179000
3,2023,Automatique,Diesel,22499.5,Peugeot,5008,5,Oui,8,Excellent,349000
4,2021,Automatique,Diesel,77499.5,Kia,Sportage,5,Oui,12,Très bon,685000


**Les colonnes disponibles**

In [ ]:
df.columns

Index(['annee', 'boite', 'carburant', 'kilometrage', 'marque', 'modele',
       'nombre_portres', 'premiere_main', 'puissance_fiscale', 'etat', 'prix'],
      dtype='object')

**Voir s'il y a des données manquantes**

In [ ]:
df.isnull().sum()

annee                0
boite                0
carburant            0
kilometrage          0
marque               0
modele               0
nombre_portres       0
premiere_main        0
puissance_fiscale    0
etat                 0
prix                 0
dtype: int64

**Examen des données**

In [ ]:
print(f"Dimensions des données : {df.shape}")
print(f"Statistiques des prix : Min={df['prix'].min()}, Max={df['prix'].max()}, Moyenne={df['prix'].mean()}")
print(f"Colonnes : {df.columns.tolist()}")

Dimensions des données : (68415, 11)
Statistiques des prix : Min=5200, Max=1000000, Moyenne=138682.62904333844
Colonnes : ['annee', 'boite', 'carburant', 'kilometrage', 'marque', 'modele', 'nombre_portres', 'premiere_main', 'puissance_fiscale', 'etat', 'prix']


**Gestion des valeurs aberrantes (AVANT la division des données)**

In [ ]:
Q1 = df['prix'].quantile(0.25)
Q3 = df['prix'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"Filtrage des prix entre {lower_bound} et {upper_bound}")
df_filtered = df[(df['prix'] >= lower_bound) & (df['prix'] <= upper_bound)]
print(f"Valeurs aberrantes supprimées : {df.shape[0] - df_filtered.shape[0]}")

Filtrage des prix entre -95505.0 et 332503.0
Valeurs aberrantes supprimées : 4041


**Sélection des caractéristiques et prétraitement**

In [ ]:
# Sélectionner les nouvelles variables
features = ['annee', 'kilometrage', 'puissance_fiscale', 'etat', 'marque', 'modele','boite','carburant']
target = 'prix'
    
# Création de X et y
X = df_filtered[features]
y = df_filtered[target]
    
# Encodage one-hot (utilisation de drop_first pour éviter la multicolinéarité)
X = pd.get_dummies(X, drop_first=True)
    
# Suppression des lignes avec des valeurs manquantes
X = X.dropna()
y = y[X.index]

**Division des données en ensembles d'entraînement et de test**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**Division des données en ensembles d'entraînement et de test**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**Mise à l'échelle des caractéristiques (IMPORTANT)**

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

**Création et entraînement du modèle (UNE SEULE FOIS)**

In [ ]:
# Créer le modèle
model = LinearRegression()

# Entraîner le modèle
model.fit(X_train_scaled, y_train)

LinearRegression()

In [ ]:
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

**Prédictions avec les données de test mises à l'échelle**

In [ ]:
y_pred = model.predict(X_test_scaled)

**Évaluation du modèle**

In [ ]:
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

# Approximation de l'accuracy (pas une véritable accuracy)
accuracy_approx = 100 - rmse
    
print("\nPerformance du modèle :")
print(f"Erreur absolue moyenne (MAE) : {mae:.2f}")
print(f"Erreur quadratique moyenne (RMSE) : {rmse:.2f}")
print(f"R² (coefficient de détermination) : {r2:.4f}")
print(f"Approximation de l'Accuracy: {accuracy_approx}%")


Performance du modèle :
Erreur absolue moyenne (MAE) : 20524.15
Erreur quadratique moyenne (RMSE) : 31435.60
R² (coefficient de détermination) : 0.8055
Approximation de l'Accuracy: -31335.599311250873%


In [ ]:
import joblib

# Sauvegarder le scaler
joblib.dump(scaler, 'pkl-files/linear_regression_scaler.pkl')

# Sauvegarder le modèle
joblib.dump(model, 'pkl-files/linear_regression_model.pkl')

# Sauvegarder les colonnes
joblib.dump(X.columns, 'pkl-files/linear_regression_columns.pkl')

print("Le scaler et le modèle de régression linéaire ont été sauvegardés sous forme de fichiers .pkl.")

Le scaler et le modèle de régression linéaire ont été sauvegardés sous forme de fichiers .pkl.
